# RAGAS

> [RAGAS](https://docs.ragas.io/en/stable/getstarted/) 是一个用于评估 RAG 系统的框架，它提供了一系列的工具，可以显著提升LLM应用的评估效率。

## 为什么需要评估RAG？

RAG系统从外部知识库中检索信息，组装上下文，再回答用户问题，不仅降低了LLM的“幻觉”问题，还提升的答案的质量。但如果我们想让RAG生成答案的质量再高一点，就会遇到一个问题，我怎么改进RAG系统，哪些改进可以使RAG生成的答案更准确呢？RAG系统包含：提示词、分块、索引、嵌入、查询、检索、生成等众多组件/环节，哪个环节的改动才能使RAG生成答案的质量更高呢？这需要一个评估标准来帮助我们定量分析。RAGAS就是这样一个评估框架用于客观评估RAG的生成质量。

## 本章内容

你将在本章学习RAGAS的基础知识并构建一个简单的RAG评估管道，实现从测试集生成->RAG评估的完整流程。

## 项目结构
本章的文件结构如下：
```
rag_eval/
|-- evals
|   |-- datasets # 数据集
|   |-- experiments # 测试结果
|   |-- logs # 测试日志 
|-- env.example # 配置文件示例
|-- evals.py # 评估脚本
|-- gen_pipline.py # 测试用例生成脚本
|-- rag.py # 简单的RAG系统
|-- pyproject.toml # 项目配置文件
|-- ragas_demo.ipynb # 教程
```

## 安装依赖&配置环境变量
```
cd rag_eval
uv sync
cp .env.example .env
```
填写 `.env` 文件中的LLM API KEY

## 构建测试集用于RAG评估
构建测试集是一个非常困难的步骤，因为不同行业面对的数据格式、规模、结构都不同，关注的点也不同，这导致难以评估测试集的好坏。本节使用的是相对简单的数据集用于演示，实际应用中需要根据实际情况构建测试集。

1. 数据集介绍<br>
本章教学使用`all-in-rag`的第一章教程

2. 加载数据集


In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

path = "./evals/datasets/custom"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()
len(docs)

3. 初始化模型

In [1]:
import os
from ragas import DiskCacheBackend
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
if load_dotenv(".env"):
    print("Loaded env")
    ENV_LOADED = True
else:
    print("No env loaded")
    ENV_LOADED = False

if ENV_LOADED:
    cache = DiskCacheBackend()
    llm = ChatOpenAI(
        model=os.getenv("EVAL_LLM_MODEL"),
        base_url=os.getenv("EVAL_LLM_BINDING_HOST"),
        api_key=os.getenv("EVAL_LLM_BINDING_API_KEY"),
        temperature=1.0,
    )
    llm = LangchainLLMWrapper(llm,cache=cache)
    embeddings = OpenAIEmbeddings(
        model = os.getenv("EVAL_EMBEDDING_MODEL"),
        base_url=os.getenv("EVAL_EMBEDDING_BINDING_HOST"),
        api_key=os.getenv("EVAL_EMBEDDING_BINDING_API_KEY")
    )
    embeddings = LangchainEmbeddingsWrapper(embeddings,cache=cache)

c:\ProgramData\anaconda3\envs\hello-agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded env


C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\206085754.py:23: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm = LangchainLLMWrapper(llm,cache=cache)
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\206085754.py:29: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embeddings = LangchainEmbeddingsWrapper(embeddings,cache=cache)


4. 设置不同角色
- RAGAS支持使用`Persona`类定义不同的角色，不同角色对生成答案的关注点不同，这有助于扩大测试用例的覆盖面。
    ```
    class Persona(BaseModel):
        name: str
        role_description: str
    ```


In [ ]:
from ragas.testset.persona import Persona

# 设定角色
personas = [
    Persona(
        name="大学生",
        role_description="在校大学生，没有AI经验，想要入门RAG",
    ),
    Persona(
        name="AI应用工程师",
        role_description="AI应用工程师，有三年工作经验，想要了解RAG的技术原理",
    )
]

5. 初始化测试用例生成器

In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(
    llm=llm, embedding_model=embeddings, persona_list=personas
)

6. 配置查询类型并适应目标语言<br>
在这里，我们加载所需的查询类型，并将其适配到目标语言。
- `SingleHopSpecificQuerySynthesizer`类是一个单跳问答生成策略，单跳问答是指对于一个简单的事实类问题,可以将知识图谱中存在的与问题实体直接相连的独立3元组作为答案。
- `MultiHopSpecificQuerySynthesizer`类是一个多跳问答生成策略，多跳问答指问答系统需要在知识图谱中的多个相关联的3元组之间执行推理才能获得正确答案.

In [ ]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)
from ragas.testset.synthesizers.multi_hop.specific import MultiHopSpecificQuerySynthesizer

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=llm), 0.5),
    (MultiHopSpecificQuerySynthesizer(llm=llm), 0.5),
]

# 使生成的测试用例为中文。其原理是在LLM的提示词中加入指定的目标语言类型，利用LLM输出，但实际上依旧使用英文输出
for query, _ in distribution:
    prompts = await query.adapt_prompts("chinese", llm=llm)
    query.set_prompts(**prompts)

7. 生成并保存测试用例

In [ ]:
dataset = generator.generate_with_langchain_docs(
    docs[:],
    testset_size=10, # 生成的测试用例数量
    query_distribution=distribution,
)
dataset = dataset.to_pandas()
dataset.to_csv(
    "./evals/datasets/testcase.csv",
    index=False,
    encoding='utf-8'
)

9. 创建`GeneratePipeline`类<br>
为了实现可复用的测试用例生成管道，我们创建一个`GeneratePipeline`类，可执行批量的测试用例生成任务，完整代码详见`gen_pipeline.py`文件。


In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import warnings
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from ragas.llms import LangchainLLMWrapper
from ragas.cache import DiskCacheBackend
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator
from ragas.testset.persona import Persona
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer
from ragas.testset.synthesizers.multi_hop.specific import MultiHopSpecificQuerySynthesizer
import traceback
import logging
 
warnings.filterwarnings("ignore")
if load_dotenv(".env"):
    print("Loaded environment variables from .env file")
    ENV_LOADED = True
else:
    print("No .env file found")
    ENV_LOADED = False

# 设置日志级别：INFO
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

class GeneratePipeline():
    def __init__(self,input_dir:str,output_dir:str='evals/experiments'):
        if ENV_LOADED:
            cache = DiskCacheBackend()
            llm = ChatOpenAI(
                model=os.getenv("EVAL_LLM_MODEL"),
                base_url=os.getenv("EVAL_LLM_BINDING_HOST"),
                api_key=os.getenv("EVAL_LLM_BINDING_API_KEY"),
                temperature=1.0,
            )
            embeddings = OpenAIEmbeddings(
                model = os.getenv("EVAL_EMBEDDING_MODEL"),
                base_url=os.getenv("EVAL_EMBEDDING_BINDING_HOST"),
                api_key=os.getenv("EVAL_EMBEDDING_BINDING_API_KEY")
            )
            self.llm = LangchainLLMWrapper(llm, cache=cache)
            self.embeddings = LangchainEmbeddingsWrapper(embeddings, cache=cache)
            self.input_dir = input_dir
            if os.path.exists(output_dir):
                self.output_dir = output_dir
            else:
                os.makedirs(output_dir)
                self.output_dir = output_dir
            self._display_configuration()
        else:
            raise Exception("Please load environment variables from .env file")
        
    def _display_configuration(self):
        # 打印模型配置信息
        logger.info("Model Configuration:")
        logger.info(f"LLM: {self.llm}")
        logger.info(f"Embeddings: {self.embeddings}")
        # 打印文件配置信息
        logger.info("File Configuration:")
        logger.info(f"Input Directory: {self.input_dir}")
        logger.info(f"Output Directory: {self.output_dir}")

    async def _load_dataset(self,file_type:str='txt'):
        loader = DirectoryLoader(self.input_dir, glob=f"**/*.{file_type}") # 实际应用中数据格式多样，需要不同的数据加载策略
        docs = loader.load()
        logger.info(f"Loaded {len(docs)} documents")
        return docs
    
    async def _gen_rag_testcase(
            self,
            docs,
            single_rate:float=0.5,
            multi_rate:float=0.5,
            language:str='chinese',
    ):
        # 安全判断
        if not os.path.exists(self.input_dir):
            raise Exception(f"Input directory {self.input_dir} does not exist")
        if single_rate < 0.0 or single_rate > 1.0 or multi_rate < 0.0 or multi_rate > 1.0:
            raise Exception("Rate must be between 0.0 and 1.0")
        if single_rate + multi_rate != 1.0:
            raise Exception("Single rate and multi rate must sum up to 1.0")
        if not docs:
            raise Exception("No documents loaded. Please run _load_dataset() before generating test cases")
        # 设定角色
        personas = [
            Persona(
                name="大学生",
                role_description="在校大学生，没有AI经验，想要入门RAG",
            ),
            Persona(
                name="AI应用工程师",
                role_description="AI应用工程师，有三年工作经验，想要了解RAG的技术原理",
            )
        ]
        # 单跳/多跳查询问题比例设置
        if single_rate == 1.0:
            distribution = [
                (SingleHopSpecificQuerySynthesizer(llm=self.llm), 1.0),
            ]
        elif multi_rate == 1.0:
            distribution = [
                (MultiHopSpecificQuerySynthesizer(llm=self.llm), 1.0),
            ]
        else:
            distribution = [
                (SingleHopSpecificQuerySynthesizer(llm=self.llm),single_rate),
                (MultiHopSpecificQuerySynthesizer(llm=self.llm), multi_rate),
            ]
        for query, _ in distribution:
            # 适配中文提示词，提升生成任务的中文场景贴合度
            prompts = await query.adapt_prompts(language, llm=self.llm)
            query.set_prompts(**prompts)

        generator = TestsetGenerator(
            llm=self.llm,
            embedding_model=self.embeddings,
            persona_list=personas
        )
        try:
            dataset = generator.generate_with_langchain_docs(
                docs[:],
                testset_size=10, # 生成的测试用例数量
                query_distribution=distribution,
            )
            dataset = dataset.to_pandas()
            dataset.to_csv(
                f"{self.output_dir}/testcase.csv",
                index=False,
                encoding="utf-8",
            )
        except Exception as e:
            logger.error(f"Error generating test cases: {e}")
            logger.error(traceback.format_exc())

## 构建RAG评估系统

### 常见的RAG评估指标

#### 上下文精确度

>上下文精确度用于评估检索器在给定查询的检索上下文中，将相关数据块的排名高于不相关数据块的能力。具体来说，它评估的是检索上下文中的相关数据块在排名中处于顶部的程度，这类似于`重排序`的功能，将高语义相关的数据块排在顶部，低语义相关的数据块排在底部。它是通过计算上下文中每个数据块的 precision@k 的平均值得得到的。Precision@k 是排名为 k 的相关数据块的数量与排名为 k 的数据块总数的比值。

- 公式
$$
\text{Context Precision@}K = \frac{\sum_{k=1}^{K} \left( \text{Precision@}k \times v_k \right)}{\text{Total number of relevant items in the top } K \text{ results}}
$$

$$
\text{Precision@}k = \frac{\text{true positives@}k}{(\text{true positives@}k + \text{false positives@}k)}
$$
其中，$K$ 是块的总数，`retrieved_contexts` 并且 $v_k \in \{0,1\}$ 是排名相关性指标 $k$。<br>
下面我们通过一个示例来了解上下文精确度的使用

In [ ]:
from ragas import evaluate
from ragas.metrics import ContextPrecision
from datasets import Dataset

# Setup LLM
llm = ChatOpenAI( 
    model=os.getenv("EVAL_LLM_MODEL"),
    base_url=os.getenv("EVAL_LLM_BINDING_HOST"),
    api_key=os.getenv("EVAL_LLM_BINDING_API_KEY"),
    temperature=1.0,
)
llm = LangchainLLMWrapper(llm)

# Create metric
metrics = [ContextPrecision()]

# Dataset
eval_dataset = Dataset.from_dict(
    {
        "question": ["Where is the Eiffel Tower located?"],
        "contexts": [["The Eiffel Tower is located in Paris.", "The Brandenburg Gate is located in Berlin."]],
        "ground_truth": ["The Eiffel Tower is located in Paris."],
    }
)
# Evaluate
eval_results = evaluate(
    dataset=eval_dataset,
    metrics=metrics,
    llm=llm,
)

df = eval_results.to_pandas()
df

C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\3743511296.py:2: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import ContextPrecision
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\3743511296.py:12: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm = LangchainLLMWrapper(llm)


Evaluating: 100%|██████████| 1/1 [00:11<00:00, 11.80s/it]


,user_input,retrieved_contexts,reference,context_precision
0,Where is the Eiffel Tower located?,"[The Eiffel Tower is located in Paris., The Br...",The Eiffel Tower is located in Paris.,1.0


#### 上下文召回率（Context Recall）
> 上下文召回率衡量的是成功检索到的相关文档（或信息片段）的数量。它侧重于不遗漏重要结果。召回率越高，意味着被遗漏的相关文档越少。简而言之，召回率关乎不遗漏任何重要内容。
- 公式
$$
\text{Context Recall} = \frac{\text{检索到的相关文档数量}}{\text{相关文档的总数}}
$$
下面通过一个示例来了解上下文召回率

In [24]:
from ragas import evaluate
from ragas.metrics import ContextRecall
from datasets import Dataset

# Setup LLM
llm = ChatOpenAI( 
    model=os.getenv("EVAL_LLM_MODEL"),
    base_url=os.getenv("EVAL_LLM_BINDING_HOST"),
    api_key=os.getenv("EVAL_LLM_BINDING_API_KEY"),
    temperature=1.0,
)
llm = LangchainLLMWrapper(llm)

# Create metric
metrics = [ContextRecall()]

# Dataset
eval_dataset = Dataset.from_dict(
    {
        "question": ["When was the first super bowl?"],  
        "answer": ["The first superbowl was held on Jan 15, 1967"],
        "retrieved_contexts":[["The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."]],
        "ground_truth":["The first superbowl was held on Jan 15, 1967"]
    }
)
# Evaluate
eval_results = evaluate(
    dataset=eval_dataset,
    metrics=metrics,
    llm=llm,
)

df = eval_results.to_pandas()
df

C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\1233587777.py:2: DeprecationWarning: Importing ContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextRecall
  from ragas.metrics import ContextRecall
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\1233587777.py:12: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm = LangchainLLMWrapper(llm)
Evaluating: 100%|██████████| 1/1 [00:07<00:00,  7.47s/it]


,user_input,retrieved_contexts,response,reference,context_recall
0,When was the first super bowl?,[The First AFL–NFL World Championship Game was...,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on Jan 15, 1967",1.0


#### 答案相关性指标
> “答案相关性”指标用于衡量RAG响应与用于查询的相关程度，其取值范围为0到1，分数越高表示与用户查询的匹配度越好。若一个答案能够直接且恰当地回应原始问题，则被认为是相关的。该指标聚焦于答案与问题意图的匹配程度，**不评估事实准确性**；同时会对不完整或包含多余细节的答案进行扣分。


-计算方式
该指标通过`user_input`（用户输入）和`response`（响应）计算，步骤如下：
1. 基于响应生成一组人工问题（默认数量为3），这些问题需体现响应的内容。
2. 计算用户输入的嵌入向量（\(E_o\)）与每个生成问题的嵌入向量（\(E_{g_i}\)）之间的余弦相似度。
3. 对这些余弦相似度分数取平均值，得到“答案相关性”。


- 公式
\[
\text{Answer Relevancy} = \frac{1}{N} \sum_{i=1}^{N} \text{cosine similarity}(E_{g_i}, E_o)
\]

\[
\text{Answer Relevancy} = \frac{1}{N} \sum_{i=1}^{N} \frac{E_{g_i} \cdot E_o}{\|E_{g_i}\| \|E_o\|}
\]


- 符号说明
- \(E_{g_i}\)：第\(i\)个生成问题的嵌入向量。
- \(E_o\)：用户输入的嵌入向量。
- \(N\)：生成问题的数量（默认值为3，可通过`strictness`参数配置）。<br>

下面通过一个示例来了解答案相关性指标的使用

In [ ]:
from ragas import evaluate
from ragas.metrics import AnswerRelevancy
from datasets import Dataset

# Setup LLM
llm = ChatOpenAI( 
    model=os.getenv("EVAL_LLM_MODEL"),
    base_url=os.getenv("EVAL_LLM_BINDING_HOST"),
    api_key=os.getenv("EVAL_LLM_BINDING_API_KEY"),
    temperature=1.0,
)
llm = LangchainLLMWrapper(llm)

embeddings = OpenAIEmbeddings(
        model = os.getenv("EVAL_EMBEDDING_MODEL"),
        base_url=os.getenv("EVAL_EMBEDDING_BINDING_HOST"),
        api_key=os.getenv("EVAL_EMBEDDING_BINDING_API_KEY")
    )
embeddings = LangchainEmbeddingsWrapper(embeddings,cache=cache)

# Create metric
metrics = [AnswerRelevancy()]

# Dataset
eval_dataset = Dataset.from_dict(
    {
        "question": ["When was the first super bowl?"],  
        "answer": ["The first superbowl was held on Jan 15, 1967"],
        "retrieved_contexts":[["The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."]],
        "ground_truth":["The first superbowl was held on Jan 15, 1967"]
    }
)
# Evaluate
eval_results = evaluate(
    dataset=eval_dataset,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings,
)

df = eval_results.to_pandas()
df

C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\773549811.py:2: DeprecationWarning: Importing AnswerRelevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import AnswerRelevancy
  from ragas.metrics import AnswerRelevancy
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\773549811.py:12: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm = LangchainLLMWrapper(llm)
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\773549811.py:19: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.em

,user_input,response,answer_relevancy
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967",NaN


#### 忠实度
> 忠实度指标用于衡量响应与检索到的上下文在事实层面的一致性。其取值范围为0到1，分数越高表明一致性越好。简单来说忠诚度用于衡量响应是否遵循检索到的上下文。
- 忠诚度和答案相关性的区别在于:如果检索到的上下文中没有包含能够回答检索问题的答案，RAG响应为“抱歉，基于现有上下文我无法回答您的问题”，此时的忠诚度等于1，而答案相关性等于0，因为响应完全是基于检索到的上下文来回答的，即便它无法准确回答检索的问题。

- 计算方法
1. 识别响应中的所有答案。
2. 检查每个答案是否能从检索到的上下文中推断出来。
3. 使用以下公式计算忠实度分数：
$$
\text{忠诚度} = \frac{\text{响应中由检索到的上下文支持的答案数量}}{\text{检索到的上下文总数量}}
$$
下面通过一个示例了解忠实度的使用

In [25]:
from ragas import evaluate
from ragas.metrics import Faithfulness
from datasets import Dataset

# Setup LLM
llm = ChatOpenAI( 
    model=os.getenv("EVAL_LLM_MODEL"),
    base_url=os.getenv("EVAL_LLM_BINDING_HOST"),
    api_key=os.getenv("EVAL_LLM_BINDING_API_KEY"),
    temperature=1.0,
)
llm = LangchainLLMWrapper(llm)

embeddings = OpenAIEmbeddings(
        model = os.getenv("EVAL_EMBEDDING_MODEL"),
        base_url=os.getenv("EVAL_EMBEDDING_BINDING_HOST"),
        api_key=os.getenv("EVAL_EMBEDDING_BINDING_API_KEY")
    )
embeddings = LangchainEmbeddingsWrapper(embeddings,cache=cache)

# Create metric
metrics = [Faithfulness()]

# Dataset
eval_dataset = Dataset.from_dict(
    {
        "question": ["When was the first super bowl?"],  
        "answer": ["The first superbowl was held on Jan 15, 1967"],
        "retrieved_contexts":[["The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."]],
        "ground_truth":["The first superbowl was held on Jan 15, 1967"]
    }
)
# Evaluate
eval_results = evaluate(
    dataset=eval_dataset,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings,
)

df = eval_results.to_pandas()
df

C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\755493608.py:2: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Faithfulness
  from ragas.metrics import Faithfulness
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\755493608.py:12: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm = LangchainLLMWrapper(llm)
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\755493608.py:19: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings 

Evaluating: 100%|██████████| 1/1 [00:12<00:00, 12.22s/it]


,user_input,retrieved_contexts,response,reference,faithfulness
0,When was the first super bowl?,[The First AFL–NFL World Championship Game was...,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on Jan 15, 1967",1.0


#### 噪声敏感度
> 噪声敏感度用于衡量系统在使用相关或不相关的检索文档时给出错误响应的频率。该分数范围为0到1，数值越低表示性能越好。噪声敏感度是通过用户输入、参考信息、响应以及检索到的上下文来计算的。
- 为了评估噪声敏感性，需要对生成的回答中的每个答案进行检查，以确定其是否基于真实情况正确无误，以及是否可归因于相关（或不相关）的检索到的上下文。理想情况下，回答中的所有答案都应得到相关检索到的上下文的支持。
- 计算公式

$$\text{噪声敏感性（相关）} = \frac{|\text{回复中错误答案的总数}|}{|\text{回复中的答案总数}|}$$
下面通过一个示例来了解噪声敏感度

In [26]:
from ragas import evaluate
from ragas.metrics import NoiseSensitivity
from datasets import Dataset

# Setup LLM
llm = ChatOpenAI( 
    model=os.getenv("EVAL_LLM_MODEL"),
    base_url=os.getenv("EVAL_LLM_BINDING_HOST"),
    api_key=os.getenv("EVAL_LLM_BINDING_API_KEY"),
    temperature=1.0,
)
llm = LangchainLLMWrapper(llm)

embeddings = OpenAIEmbeddings(
        model = os.getenv("EVAL_EMBEDDING_MODEL"),
        base_url=os.getenv("EVAL_EMBEDDING_BINDING_HOST"),
        api_key=os.getenv("EVAL_EMBEDDING_BINDING_API_KEY")
    )
embeddings = LangchainEmbeddingsWrapper(embeddings,cache=cache)

# Create metric
metrics = [NoiseSensitivity()]

# Dataset
eval_dataset = Dataset.from_dict(
    {
        "question": ["When was the first super bowl?"],  
        "answer": ["The first superbowl was held on Jan 15, 1967"],
        "retrieved_contexts":[["The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."]],
        "ground_truth":["The first superbowl was held on Jan 15, 1967"]
    }
)
# Evaluate
eval_results = evaluate(
    dataset=eval_dataset,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings,
)

df = eval_results.to_pandas()
df

C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\397696283.py:2: DeprecationWarning: Importing NoiseSensitivity from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import NoiseSensitivity
  from ragas.metrics import NoiseSensitivity
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\397696283.py:12: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm = LangchainLLMWrapper(llm)
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\397696283.py:19: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas

,user_input,retrieved_contexts,response,reference,noise_sensitivity(mode=relevant)
0,When was the first super bowl?,[The First AFL–NFL World Championship Game was...,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on Jan 15, 1967",0.0


#### 上下文实体召回率

> 上下文实体召回率（Context Entity Recall） 指标用于衡量检索到的上下文的召回程度，其计算基于同时存在于“参考文本”和“检索到的上下文”中的实体数量，与仅存在于“参考文本”中的实体数量的相对比例。简单来说，它衡量的是从参考文本中召回的实体占比。

- 该指标适用于基于事实的场景（例如旅游服务台、历史问答等），可帮助评估实体检索机制——因为在实体很重要的场景中，我们需要检索到的上下文能覆盖这些实体。该指标主要用于融合`知识图谱`的高级RAG系统，例如：`GraphRAG``LightRAG``RAGFlow``KAG`等。


- 计算该指标需用到两个集合：
    - \( RE \)：参考文本中的实体集合。
    - \( RCE \)：检索到的上下文中的实体集合。


我们先计算两个集合的公共实体数量（ RCE和RE ），再将其除以参考文本中的实体总数（RE）。公式为：
$$\text{上下文实体召回率} = \frac{\text{RE与RCE的公共实体数量}}{\text{RE中的实体总数}}$$
下面我们通过一个示例来了解上下文实体召回率的使用

In [27]:
from ragas import evaluate
from ragas.metrics import ContextEntityRecall
from datasets import Dataset

# Setup LLM
llm = ChatOpenAI( 
    model=os.getenv("EVAL_LLM_MODEL"),
    base_url=os.getenv("EVAL_LLM_BINDING_HOST"),
    api_key=os.getenv("EVAL_LLM_BINDING_API_KEY"),
    temperature=1.0,
)
llm = LangchainLLMWrapper(llm)

embeddings = OpenAIEmbeddings(
        model = os.getenv("EVAL_EMBEDDING_MODEL"),
        base_url=os.getenv("EVAL_EMBEDDING_BINDING_HOST"),
        api_key=os.getenv("EVAL_EMBEDDING_BINDING_API_KEY")
    )
embeddings = LangchainEmbeddingsWrapper(embeddings,cache=cache)

# Create metric
metrics = [ContextEntityRecall()]

# Dataset
eval_dataset = Dataset.from_dict(
    {
        "question": ["When was the first super bowl?"],  
        "answer": ["The first superbowl was held on Jan 15, 1967"],
        "retrieved_contexts":[["The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."]],
        "ground_truth":["The first superbowl was held on Jan 15, 1967"]
    }
)
# Evaluate
eval_results = evaluate(
    dataset=eval_dataset,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings,
)

df = eval_results.to_pandas()
df

C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\177973621.py:2: DeprecationWarning: Importing ContextEntityRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextEntityRecall
  from ragas.metrics import ContextEntityRecall
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\177973621.py:12: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm = LangchainLLMWrapper(llm)
C:\Users\charlotte\AppData\Local\Temp\ipykernel_59100\177973621.py:19: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or f

,user_input,retrieved_contexts,response,reference,context_entity_recall
0,When was the first super bowl?,[The First AFL–NFL World Championship Game was...,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on Jan 15, 1967",0.0


### 自动化的RAG评估管道
我们将在本小节实现一个通用的支持多并发的RAG评估管道，完整代码详见`evals_pipeline.py`，它依赖于已经构建好的简易RAG系统`rag.py`。由于本章的重点在于如何评估RAG效果，因此关于这个简易RAG系统实现的细节不展开描述。

#### 类结构设计

In [ ]:
from pathlib import Path
class RAGEvalPipeline:
    def __init__(self,  test_dataset_path: str = "evals/experiments/testcase.csv"):
        if ENV_LOADED:
            # 初始化模型
            cache = DiskCacheBackend()
            llm = ChatOpenAI( 
                model=os.getenv("EVAL_LLM_MODEL"),
                base_url=os.getenv("EVAL_LLM_BINDING_HOST"),
                api_key=os.getenv("EVAL_LLM_BINDING_API_KEY"),
                temperature=1.0,
            )
            self.chat_model = llm
            self.llm = LangchainLLMWrapper(llm, cache=cache)
            embeddings = OpenAIEmbeddings(
                model = os.getenv("EVAL_EMBEDDING_MODEL"),
                base_url=os.getenv("EVAL_EMBEDDING_BINDING_HOST"),
                api_key=os.getenv("EVAL_EMBEDDING_BINDING_API_KEY")
            )
            self.embeddings = LangchainEmbeddingsWrapper(embeddings, cache=cache)
            self.eval_max_retries = int(os.getenv("EVAL_MAX_RETRIES", "5"))
            self.eval_timeout = int(os.getenv("EVAL_TIMEOUT", "60"))
            self.test_dataset_path = Path(test_dataset_path)
            self.results_dir = Path(__file__).parent / "results"
            self.results_dir.mkdir(exist_ok=True)

            # Add FileHandler
            log_file = self.results_dir / "eval_pipeline.log"
            fh = logging.FileHandler(log_file, encoding='utf-8')
            fh.setLevel(logging.INFO)
            fh.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
            logger.addHandler(fh)

            self.test_cases = self._load_test_dataset()
            self._display_configuration()
        else:
            raise Exception("Please load environment variables from .env file")

#### 加载测试集

In [ ]:
def _load_test_dataset(self):
        if not self.test_dataset_path.exists():
            raise FileNotFoundError(f"Test dataset not found: {self.test_dataset_path}")
        df = pd.read_csv(self.test_dataset_path)
        # 校验字段
        must_columns = ['user_input','reference_contexts','reference']
        for col in must_columns:
            if col not in df.columns:
                raise ValueError(f"Missing column: {col}")
        return df

#### 生成RAG响应

In [ ]:
from rag import default_rag_client
import asyncio
async def gen_rag_response(self):
        rag_logs_dir = self.results_dir / "logs"
        rag_logs_dir.mkdir(exist_ok=True)
        rag_client = default_rag_client(llm_client=self.chat_model, logdir=str(rag_logs_dir))

        top_k = 5

        async def run_case(row):
            question = row["user_input"]
            last_error = None
            for attempt in range(self.eval_max_retries):
                try:
                    result = await asyncio.wait_for(
                        rag_client.query(question, top_k=top_k),
                        timeout=self.eval_timeout
                    )
                    answer = result.get("answer", "")
                    # 如果返回的是错误信息则重试
                    if isinstance(answer, str) and answer.startswith("Error"):
                        last_error = answer
                        continue
                    return {
                        "user_input": question,
                        "reference_contexts": row.get("reference_contexts", ""),
                        "reference": row.get("reference", ""),
                        "answer": answer,
                        "run_id": result.get("run_id", ""),
                    }
                except Exception as e:
                    last_error = str(e)
                    continue
            # 重试失败后返回错误
            return {
                "user_input": question,
                "reference_contexts": row.get("reference_contexts", ""),
                "reference": row.get("reference", ""),
                "answer": f"Error after retries: {last_error}",
                "run_id": "",
            }

        tasks = [run_case(row) for _, row in self.test_cases.iterrows()]
        results = await asyncio.gather(*tasks)
        df = pd.DataFrame(results)
        return df

#### 评估RAG生成质量
- eval_rag_response 评估RAG生成质量的主函数
- eval_single_response 评估单个RAG生成的答案
- worker 异步任务函数

In [ ]:
import datetime
async def eval_rag_response(self,eval_dataset:pd.DataFrame):
        must_columns = ['user_input','reference_contexts','reference','answer']
        for col in must_columns:
            if col not in eval_dataset.columns:
                raise ValueError(f"Missing column: {col}")
        
        # 准备数据，转换为 Ragas Dataset 格式
        data = {
            "question": eval_dataset["user_input"].tolist(),
            "answer": eval_dataset["answer"].tolist(),
            "ground_truth": eval_dataset["reference"].tolist(),
            "retrieved_contexts": [],
        }
        
        for _, row in eval_dataset.iterrows():
            ctx = row["reference_contexts"]
            if isinstance(ctx, str):
                data["retrieved_contexts"].append([ctx])
            elif isinstance(ctx, list):
                data["retrieved_contexts"].append(ctx)
            else:
                data["retrieved_contexts"].append([str(ctx)] if ctx is not None else [])

        dataset = Dataset.from_dict(data)

        metrics = [
            ContextPrecision(), 
            ContextRecall(), 
            AnswerRelevancy(), 
            ContextEntityRecall(), 
            NoiseSensitivity(), 
            Faithfulness() 
        ]
        
        logger.info("Starting bulk evaluation with Ragas...")
        
        # 使用 asyncio.to_thread 避免阻塞主事件循环，因为 evaluate 可能是同步调用的
        result = await asyncio.to_thread(
            evaluate,
            dataset=dataset,
            metrics=metrics,
            llm=self.llm,
            embeddings=self.embeddings,
            raise_exceptions=False
        )
        
        out_df = result.to_pandas()
        
        outfile = self.results_dir / f"rag_eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        out_df.to_csv(outfile, index=False, encoding="utf-8")
        logger.info("Saved RAG eval results to %s", outfile)
        return out_df

#### 结果展示

In [ ]:
import numpy as np
def _display_results_table(self, results: pd.DataFrame):
        logger.info("")
        logger.info("%s", "=" * 115)
        logger.info("📊 EVALUATION RESULTS SUMMARY")
        logger.info("%s", "=" * 115)

        logger.info(
            "%-4s | %-50s | %6s | %7s | %6s | %7s | %6s | %6s | %6s | %6s",
            "#",
            "Question",
            "Faith",
            "AnswRel",
            "CtxRec",
            "CtxPrec",
            "EntRec",
            "Noise",
            "RAGAS",
            "Status",
        )
        logger.info("%s", "-" * 115)

        def fmt(val, width):
            try:
                if val is None or (isinstance(val, float) and (pd.isna(val) or val != val)) or (hasattr(np, "isnan") and np.isnan(val)):
                    return f"{'N/A':>{width}}"
                return f"{float(val):>{width}.3f}"
            except Exception:
                return f"{str(val)[:width]:>{width}}"

        for idx, row in results.iterrows():
            q = row.get("question", row.get("user_input", ""))
            q_disp = (q[:47] + "...") if isinstance(q, str) and len(q) > 50 else q
            is_error = row.get("status", "") == "error"
            if is_error:
                err = row.get("error", "Unknown error")
                err_disp = (err[:20] + "...") if isinstance(err, str) and len(err) > 23 else err
                logger.info(
                    "%-4d | %-50s | %6s | %7s | %6s | %7s | %6s | %6s | %6s | ✗ %s",
                    idx + 1,
                    q_disp,
                    "N/A",
                    "N/A",
                    "N/A",
                    "N/A",
                    "N/A",
                    "N/A",
                    "N/A",
                    err_disp,
                )
                continue

            faith = row.get("faithfulness", None)
            ans_rel = row.get("answer_relevancy", row.get("answer_relevance", None))
            ctx_rec = row.get("context_recall", None)
            ctx_prec = row.get("context_precision", None)
            ent_rec = row.get("context_entity_recall", None)
            noise = row.get("noise_sensitivity", None)
            ragas = row.get("ragas_score", None)
            logger.info(
                "%-4d | %-50s | %s | %s | %s | %s | %s | %s | %6s | %s",
                idx + 1,
                q_disp,
                fmt(faith, 6),
                fmt(ans_rel, 7),
                fmt(ctx_rec, 6),
                fmt(ctx_prec, 7),
                fmt(ent_rec, 6),
                fmt(noise, 6),
                fmt(ragas, 6),
                "✓",
            )

        logger.info("%s", "=" * 115)